In [11]:
import numpy as np
# import impl.neuralnet as nn
from impl.solver import *
import sys

In [12]:
n_iter = 1000 # number of epochs
alpha = 1e-3
mb_size = 64 # minibatch size usually compatible to the Cache/RAM size
n_experiment = 1
reg = 1e-5
print_after = 100
p_dropout = 0.8 # dropout/keep_prob
loss = 'cross_ent'
nonlin = 'relu'
solver = 'sgd'

In [13]:
# import hipsternet.input_data as input_data  # NOT used for MNIST
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('data/MNIST_data/', one_hot=False)
X_train, y_train = mnist.train.images, mnist.train.labels
X_val, y_val = mnist.validation.images, mnist.validation.labels
X_test, y_test = mnist.test.images, mnist.test.labels
y_test.shape, y_val.shape, y_train.shape

Extracting data/MNIST_data/train-images-idx3-ubyte.gz
Extracting data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting data/MNIST_data/t10k-labels-idx1-ubyte.gz


((10000,), (5000,), (55000,))

In [14]:
M, D, C = X_train.shape[0], X_train.shape[1], y_train.max() + 1
M, D, C

(55000, 784, 10)

In [15]:
import hipsternet.utils as utils
X_train, X_val, X_test = utils.prepro(X_train, X_val, X_test)
X_train.shape, X_val.shape, X_test.shape

((55000, 784), (5000, 784), (10000, 784))

In [16]:
# # if net_type == 'cnn':
# img_shape = (1, 28, 28)
# X_train = X_train.reshape(-1, *img_shape)
# X_val = X_val.reshape(-1, *img_shape)
# X_test = X_test.reshape(-1, *img_shape)
# X_train.shape, X_val.shape, X_test.shape
# # ((55000, 1, 28, 28), (5000, 1, 28, 28), (10000, 1, 28, 28))
# # In [19]:

In [17]:
solvers = dict(
    sgd=sgd,
    momentum=momentum,
    nesterov=nesterov,
    adagrad=adagrad,
    rmsprop=rmsprop,
    adam=adam
)
solvers

{'adagrad': <function hipsternet.solver.adagrad>,
 'adam': <function hipsternet.solver.adam>,
 'momentum': <function hipsternet.solver.momentum>,
 'nesterov': <function hipsternet.solver.nesterov>,
 'rmsprop': <function hipsternet.solver.rmsprop>,
 'sgd': <function hipsternet.solver.sgd>}

In [18]:
solver_fun = solvers[solver] # solver functions
accs = np.zeros(n_experiment)
solver_fun, accs

(<function hipsternet.solver.sgd>, array([ 0.]))

In [19]:
print()
print('Experimenting on {}'.format(solver))
print()


Experimenting on sgd



In [20]:
import hipsternet.FFNN as ffnn # Feed Forward Neural Net
import hipsternet.NN as nn_

for k in range(n_experiment):
#     print('PReLU as the activation function')
    print('ReLU as the activation function')
#     print('Softplus as the activation function')
#     print('Sigmoid as the activation function')
#     print('ELU as the activation function')
#     print('Neural_Biased_Fully_Parametric_leaky_ReLU as the activation function')
#     print('Biased_Fully_Parametric_ReLU as the activation function')
#     print('Fully_Parametric_ReLU as the activation function')
#     print('P_Leaky ReLU as the activation function')
#     print('Leaky ReLU as the activation function')
#     print('Noisy ReLU as the activation function')
#     print('Tanh as the activation function')
#     print('Integral Tanh as the activation function')
    print('Experiment-{}'.format(k + 1))

    # net = nn.FeedForwardNet(D, C, H=128, lam=reg, p_dropout=p_dropout, loss=loss, nonlin=nonlin)
    net = ffnn.FFNN(D, C, H=128, lam=reg, p_dropout=p_dropout, loss=loss, nonlin=nonlin)
    #     net = nn.ConvNet(D=10, C=C, H=128) # Original one
#     net = ConvNet_(C=C, D=10, H=128) # Mine
    

    net = solver_fun(nn=net, X_train=X_train, y_train=y_train, val_set=(X_val, y_val), 
                     mb_size=mb_size, alpha=alpha, n_iter=n_iter, print_after=print_after)

    y_pred = net.predict(X_test)
    accs[k] = np.mean(y_pred == y_test)

print()
print('Test Mean accuracy: {:.4f}, std: {:.4f}'.format(accs.mean(), accs.std()))

Neural_Biased_Fully_Parametric_leaky_ReLU as the activation function
Experiment-1
Iter-100 loss: 2.1331 validation: 0.226200
Iter-200 loss: 2.1931 validation: 0.357200
Iter-300 loss: 1.7955 validation: 0.465600
Iter-400 loss: 1.8817 validation: 0.549200
Iter-500 loss: 1.5905 validation: 0.607600
Iter-600 loss: 1.5075 validation: 0.666600
Iter-700 loss: 1.2493 validation: 0.701200
Iter-800 loss: 1.0984 validation: 0.725200
Iter-900 loss: 1.1242 validation: 0.744200
Iter-1000 loss: 1.2582 validation: 0.757800

Test Mean accuracy: 0.7553, std: 0.0000


# Future work: -----------------